In [37]:
import numpy as np
from sklearn.svm import OneClassSVM as OCSVM
import csv
from pathlib import Path
import collections
from numpy import savetxt
import matplotlib.pyplot as plt
import matplotlib.font_manager
from sklearn import svm
from matplotlib.pyplot import figure
import pandas as pd
import time

In [38]:
fold = 'Fold1/'

In [12]:
lista_VAE = list()         
basepath = Path('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/SVAE/' + fold)
files_in_basepath = basepath.iterdir()
for item in files_in_basepath:
  if item.is_file():
    arq = open('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/SVAE/'+ fold + item.name, 'r')
    mat_embedding = list()
    reader = csv.reader(arq)
    for doc in reader:
      mat_embedding.append([float(i) for i in doc])
    lista_VAE.append((np.array(mat_embedding),item.name.replace('.csv','')))

In [ ]:
lista_OCSVM = list()         
basepath = Path('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/DBERT-WITH-OTHER-CLASSES/' + fold)
files_in_basepath = basepath.iterdir()
for item in files_in_basepath:
  if item.is_file():
    arq = open('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/DBERT-WITH-OTHER-CLASSES/' + fold + item.name, 'r')
    mat_embedding = list()
    reader = csv.reader(arq)
    for doc in reader:
      mat_embedding.append([float(i) for i in doc])
    lista_OCSVM.append((np.array(mat_embedding),item.name.replace('.csv','')))
    arq.close()

In [64]:
lista_Bow = list()
basepath = Path('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/Bow/TFIDF/' + fold)
files_in_basepath = basepath.iterdir()
for item in files_in_basepath:
  if item.is_file():
    df = pd.read_csv('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/Bow/TFIDF/' + fold + item.name, 'r',delimiter=',')
    lista_Bow.append((df,item.name.replace('.csv','')))

In [40]:
def evaluate_model(model, X_train, X_test, X_ruido):
  
  model.fit(X_train)

  Y_pred_interest = model.predict(X_test)
  
  Y_pred_ruido = model.predict(X_ruido)

  counter_interest = collections.Counter(Y_pred_interest)

  counter_ruido = collections.Counter(Y_pred_ruido)

  vp = counter_interest[1] # VP
  fn = counter_interest[-1] # FN
  fp = counter_ruido[1] # FP
  vn = counter_ruido[-1] # VN
  
  if vp ==0: 
    precisao = 0
    revocacao = 0
    f1 = 0
  else:  
    precisao = vp / (vp + fp) # VP / (VP + FP)
    revocacao = vp / (vp + fn) # VP / (VP + FN)
    f1 = (2 * precisao * revocacao) / (precisao + revocacao) 

  
  sfp = str(fp)
  if len(sfp) == 3: sfp+= ' ' 
  if len(sfp) == 2: sfp+= '  '
  if len(sfp) == 1: sfp+= '   '
  svn = str(vn)
  if len(svn) == 3: svn+= ' ' 
  if len(svn) == 2: svn+= '  '
  if len(svn) == 1: svn+= '   '
  svp = str(vp)
  if len(svp) == 3: svp+= ' ' 
  if len(svp) == 2: svp+= '  '
  if len(svp) == 1: svp+= '   '
  sfn = str(fn)
  if len(sfn) == 3: sfn+= ' ' 
  if len(sfn) == 2: sfn+= '  '
  if len(sfn) == 1: sfn+= '   '


  print('CONFUSION MATRIX:')
  print('                                +---------+---------+')
  print('                                |    Classe Real    |')
  print('                                +---------+---------+')
  print('                                | Classe  |         |')
  print('                                |Interesse| Outlier |')
  print('+---------+---------------------+---------+---------+')
  print('| Classe  | Classe de Interesse |   '+svp+'  |   '+sfp+'  |')
  print('| Predita +---------------------|---------|---------|')
  print('| (OCSVM) | Outlier             |   '+sfn+'  |   '+svn+'  |')
  print('+---------+---------------------+---------+---------+')
  
  print('METRICS: ')
  print('Precisão: ' + str(precisao))
  print('Revocação: ' + str(revocacao))
  print('F1-Score: ' + str(f1))
  
  return precisao,revocacao,f1

In [41]:
def evaluate_models(models,preprocessing):
  
  if preprocessing == 'SVAE' or preprocessing == 'DBERT':
  
    if preprocessing == 'SVAE': lista_prepro = lista_VAE
    else: lista_prepro = lista_OCSVM
    
    for matriz,base in lista_prepro:
      print('Na Base ' + base)
      for modelo in models.keys():
        print('No modelo ' + modelo)
        treino = int(base.split('_')[2])
        teste = int(base.split('_')[3])
        start = time.time()
        pre,rev,f1 = evaluate_model(models[modelo], matriz[:treino],matriz[treino:teste],matriz[teste:])
        end = time.time()
        temp = end - start
        escreve(pre,rev,f1,base,modelo,temp,preprocessing)

  else: 
    lista_prepro = lista_Bow
    for df,base in lista_prepro:
      print('Na Base ' + base)
      for modelo in models.keys():
        print('No modelo ' + modelo)
        interesse = df[df.label=='interest']
        treino = int(len(interesse)/3)
        outlier = df[df.label=='outlier']

        start = time.time()
        pre,rev,f1 = evaluate_model(models[modelo],interesse[:treino].drop('label', axis=1),interesse[treino:].drop('label', axis=1),outlier.drop('label', axis=1))
        end = time.time()
        temp = end - start
        #escreve(pre,rev,f1,base,modelo,temp,preprocessing)

In [15]:
def escreve(pre,rev,f1,prepro,modelo,temp,preprocessing):
  nome = '/content/drive/My Drive/USP/qualis/Experimentos/Resultados/'+ preprocessing +'/'+ fold + str(prepro.split('_')[0]) + '_' + str(prepro.split('_')[1]) + '_' + str(modelo.split('_')[0]) + '.csv'

  if not Path(nome).is_file():
    arquivo = open(nome,'a')
    arquivo.write('Parametros;Precisao;Revocacao;F1-Score;Tempo\n')
    arquivo.close()
  
  arquivo = open(nome,'a')
  arquivo.write(str(prepro) + '_' + str(modelo)+';'+str("%.4f" % pre)+';'+str("%.4f" % rev)+';'+str("%.4f" % f1) +';'+str("%.4f" % temp) + '\n')
  arquivo.close()

In [66]:
experimentos_OCSVM = {
    "OCSVM_RBF_0,0001_scale" : OCSVM(nu=0.0001, kernel='rbf'),
    "OCSVM_RBF_0,0005_scale" : OCSVM(nu=0.0005, kernel='rbf'),
    "OCSVM_RBF_0,001_scale" : OCSVM(nu=0.001, kernel='rbf'),
    "OCSVM_RBF_0,005_scale" : OCSVM(nu=0.005, kernel='rbf'),
    "OCSVM_RBF_0,01_scale" : OCSVM(nu=0.01, kernel='rbf'),
    "OCSVM_RBF_0,02_scale" : OCSVM(nu=0.02, kernel='rbf'),
    "OCSVM_RBF_0,03_scale" : OCSVM(nu=0.03, kernel='rbf'),
    "OCSVM_RBF_0,04_scale" : OCSVM(nu=0.04, kernel='rbf'),
    "OCSVM_RBF_0,05_scale" : OCSVM(nu=0.05, kernel='rbf'),
    "OCSVM_RBF_0,06_scale" : OCSVM(nu=0.06, kernel='rbf'),
    "OCSVM_RBF_0,07_scale" : OCSVM(nu=0.07, kernel='rbf'),
    "OCSVM_RBF_0,08_scale" : OCSVM(nu=0.08, kernel='rbf'),
    "OCSVM_RBF_0,09_scale" : OCSVM(nu=0.09, kernel='rbf'),
    "OCSVM_RBF_0,1_scale" : OCSVM(nu=0.1, kernel='rbf'),
    "OCSVM_RBF_0,2_scale" : OCSVM(nu=0.2, kernel='rbf'),
    "OCSVM_RBF_0,3_scale" : OCSVM(nu=0.3, kernel='rbf'),
    "OCSVM_RBF_0,4_scale" : OCSVM(nu=0.4, kernel='rbf'),
    "OCSVM_RBF_0,5_scale" : OCSVM(nu=0.5, kernel='rbf'),
    "OCSVM_RBF_0,6_scale" : OCSVM(nu=0.6, kernel='rbf'),
    "OCSVM_RBF_0,7_scale" : OCSVM(nu=0.7, kernel='rbf'),
    "OCSVM_RBF_0,8_scale" : OCSVM(nu=0.8, kernel='rbf'),
    "OCSVM_RBF_0,9_scale" : OCSVM(nu=0.9, kernel='rbf'),
    "OCSVM_linear" : OCSVM(kernel='linear'),
    "OCSVM_sig" : OCSVM(kernel='sigmoid'),
    "OCSVM_poly" : OCSVM(kernel='poly')
}

In [ ]:
evaluate_models(experimentos_OCSVM,'SVAE')

In [ ]:
evaluate_models(experimentos_OCSVM,'DBERT')

In [ ]:
evaluate_models(experimentos_OCSVM,'BoW')

In [73]:
for i in range(1,11):
  basepath = Path('/content/drive/My Drive/USP/qualis/Experimentos/Resultados/DBERT/Fold'+ str(i)+'/')
  files_in_basepath = basepath.iterdir()
  count =0
  for item in files_in_basepath:
    if item.is_file():
      arq = open('/content/drive/My Drive/USP/qualis/Experimentos/Resultados/DBERT/Fold'+ str(i)+'/' + item.name, 'r')
      mat_embedding = list()
      reader = csv.reader(arq)
      count=0
      for doc in reader:
        count+=1
      if count != 23:
        print('no fold:' + str(i))
        print(item.name)